# Creating a Delft3D FM 1D2D fluvial flood model of Magdalena river 

The exercises in this notebook will serve as introduction on how to use [HYDROLIB-core](https://github.com/Deltares/HYDROLIB-core) functionalities. We will create and adjust a Delft3D FM Suite 1D2D fluvial flood model of the Magdalena river in Columbia. 

First a 1D river model of the Magdalena river and Canal del Dique will be created, shown in the left figure. This model will be extended with a 2D Flexible Mesh, with a uniform roughness field. Adding a Digital Terrain Model (DTM) on that 2D mesh is currently not part of this tutorial, but in practice this is indispensible to mimic the fluvial flood patterns.

<p align="center">
  <img alt="1D2D" src="figures/1D2D_model.png" width="45%">
</p>

Note that the model that will be created is for educational purposes and consists partially of fictive or unvalidated model data. 

## Using HYDROLIB-core to build models
HYDROLIB-core is a Python package that offers functionality to process D-HYDRO input files. It offers Python-wrapped classes that represent these files and their content. 
HYDROLIB-core serves as the basis for various pre- and postprocessing tools for a modelling workflow of hydrodynamic simulations in D-HYDRO. It can easily be used to build models and export the model files that can aftewards be run by dflowfm or other supported D-HYDRO kernels.

## Content of the tutorial

The goal of this tutorial is to familiarize modelers with applying HYDROLIB-core functionalities in their modelling procedures for building new models or adjusting existing models. As mentioned above, you will create an 1D2D fluvial flood model from scratch using the D-Flow FM HYDROLIB-core functionalities and the underlying functions from mesh generation ([MeshKernelPy](https://github.com/Deltares/MeshKernelPy)). We will walk through the following steps to build the model:

1. Selecting the data location
2. Create an empty D-Flow FM model 
3. Read branches
4. Create computational grid
5. Add cross-sections
6. Add roughness
7. Add boundary conditions
8. Add initial conditions
9.  Add a weir to the model
10. Save 1D model
11. Create 2D flexible mesh
12. Add 2D DTM to model
13. Save 1D2D model

## 💡 Tips for working in the Jupyter Notebook
  * __Tab__: Auto-complete on code
  * __Shift__+__Tab__: Tooltip
  * __Ctrl__+__Enter__: Run cell 
  * __Shift__+__Enter__: Run cell and move to next cell 
  * __Ctrl__+__Shift__+__P__: Open command palette

## Import the needed modules
Several Python packages are required to run the notebook. These packages are imported below. Please consult the [HYDROLIB-core documentation](https://deltares.github.io/HYDROLIB-core/0.5.2/) if you want to use additional HYDROLIB-core functionalities later.

In [ ]:
# General
import geopandas as gpd
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from pathlib import Path

# MeshKernel
import meshkernel as mk
from meshkernel import GeometryList, MakeGridParameters

# HYDROLIB-core
from hydrolib.core.dflowfm.mdu import FMModel, AutoStartOption
from hydrolib.core.dflowfm.net.models import *
from hydrolib.core.dflowfm.crosssection.models import CrossLocModel, CrossDefModel, YZCrsDef
from hydrolib.core.dflowfm.friction.models import *
from hydrolib.core.dflowfm.bc.models import *
from hydrolib.core.dflowfm.ext.models import *
from hydrolib.core.dflowfm.inifield.models import *
from hydrolib.core.dflowfm.onedfield.models import *
from hydrolib.core.dflowfm.structure.models import Weir, FlowDirection, StructureModel

print("Imports were successful!")


## Here starts the tutorial

### 1. Selecting the data location

__Exercise__: Create a variable that contains the Path to the ```data``` folder.

In [ ]:
root = Path.cwd()
data_dir = root / 'data'

print(f"Data location: {data_dir}")

### 2. Create an empty D-Flow FM model

To start a model from scratch, you first need to create an empty model without data. In this tutorial we will create a D-Flow FM model. The MDU file (master definition file) is the main input file of the D-Flow FM model.

A new D-Flow FM model is constructed with the `FMModel` initializer. 

__Exercise__: 
* Create a variable containing a new `FMModel` object.
* Create a variable containing the export directory 
* Export the D-Flow FM model to an MDU file inside the export directory

API references: 
* [FMModel](https://deltares.github.io/HYDROLIB-core/0.5.2/reference/mdu/#hydrolib.core.dflowfm.mdu.models.FMModel)
* [FileModel.filepath](https://deltares.github.io/HYDROLIB-core/0.5.2/reference/api/#hydrolib.core.basemodel.FileLoadContext.retrieve_model)
* [FileModel.save()](https://deltares.github.io/HYDROLIB-core/0.5.2/reference/api/#hydrolib.core.basemodel.FileModel.save)

In [ ]:
model = FMModel()
export_dir = root / '1D_model'
model.filepath = export_dir / 'Magdalena_1D.mdu'
model.save()

print(f"Model saved: {model.filepath}")

## Part 1: Setting up a 1D Delft3D FM model
In this part we will build up the complete 1D model schematization step by step. At the end you should have a complete set of model input files for a 1D calculation.

### 3. Reading the branch data from a shape file
The model will include the lower part of the Magadalena from Cordoba Teton to the river mouth Baranquilla and the Canal del Dique.

__Exercise__:
* Read the branch data from the `branches_Branches.shp` file
* Print the branch data, such that we can answer the following questions:
  * How many river branches does the model have?
  * What are their names?
  * Which friction type and friction value is used for each branch?

API references: 
* [gpd.read_file](https://geopandas.org/en/stable/docs/reference/api/geopandas.read_file.html#geopandas-read-file)

In [ ]:
# Read the branches
branch_shp_file = data_dir / 'branches_Branches.shp'
branches_gdf = gpd.read_file(filename = branch_shp_file)

print(branches_gdf)

# Answers:
# * 3 branches
# * Channel_1D_1_A, Channel_1D_1_B and Channel_1D_1
# * Channel_1D_1_A: Manning, 0.03 
#   Channel_1D_1_B: Manning, 0.02
#   Channel_1D_1  : Manning, 0.02 

### 5. Assign a 1D computational grid to the model

In the previous step we read the geometry and attributes of the branches from the shape file. This branch data will become the network input for our D-Flow FM model. 

Since the model does not have a network yet, we need to create a new `Network` object and assign it to the model. A network may contain a 1D network, a 2D mesh and 1D2D links.

**Exercise**
* Create a new network
* Assign the network to the model and assign a (relative) file path for the net file
* The new network should contain the data that we read from the shape file:
  * Three `Branch` objects will need to be added to the network. 
  * For each branch, the name, geometry and computational grid points should be provided. 
  * The computational points are not included in the data, we will need to generate those with a grid point distance as specified:

    | **Branch name** | **dx [m]** |
    |-----------------|------------|
    | Channel_1D_1_A  | 1000       |
    | Channel_1D_1_B  | 1000       |
    | Channel_1D_1    | 500        |

Hint 💡: The MDU file contains a [Geometry] section which has the `NetFile` reference, e.g.:

```
[Geometry]
netFile                    = FlowFM_net.nc
bathymetryFile             =
dryPointsFile              =
structureFile              = structures.ini
iniFieldFile               = initialFields.ini
...
```

API references: 
* [Network](https://deltares.github.io/HYDROLIB-core/0.5.2/reference/net/#hydrolib.core.dflowfm.net.models.Network)
* [NetworkModel](https://deltares.github.io/HYDROLIB-core/0.5.2/reference/net/#hydrolib.core.dflowfm.net.models.NetworkModel)
* [Branch](https://deltares.github.io/HYDROLIB-core/0.5.2/reference/net/#hydrolib.core.dflowfm.net.models.Branch)
* [Branch.generate_nodes()](https://deltares.github.io/HYDROLIB-core/0.5.2/reference/net/#hydrolib.core.dflowfm.net.models.Branch.generate_nodes)
* [Network.mesh1d_add_branch()](TODO)

In [ ]:
# Assign a new network to the D-Flow FM model and specify the file path
network = Network()
model.geometry.netfile.network = network
model.geometry.netfile.filepath = "FlowFM_net.nc"

# Specify 1D computational grid point distance per branch
dx_per_branch = {
    "Channel_1D_1_A" : 1000,
    "Channel_1D_1_B" : 1000,
    "Channel_1D_1" : 500,
}

# Assign geometry of the branches and generate the computational grid on top of the branches.
for index, branch_data in branches_gdf.iterrows():
    xy = np.transpose(branch_data.geometry.xy)    
    branch_new = Branch(geometry=xy) 
    branch_new.generate_nodes(mesh1d_edge_length=dx_per_branch[branch_data.Name])
    network.mesh1d_add_branch(branch_new, name=branch_data.Name)

network._mesh1d._set_mesh1d()

We can plot the branches onto a background:

In [ ]:
# Create a figure with the 1D network
figure, axis = plt.subplots(ncols=1)
img = plt.imread(data_dir / "osm_background.png")

# Add branch names to the plot
for branch_name, branch in network._mesh1d.branches.items():
    middle = int((len(branch.geometry) - 1)/2)
    xy = branch.geometry[middle]
    axis.text(xy[0], xy[1], branch_name, ha='center')

axis.imshow(img, extent=[440000,
                        531544.97964175534434617,
                        1057000,
                        1228644.01383191486820579])

network._mesh1d._get_mesh1d().plot_edges(ax=axis, color='blue')

### 6. Add cross-sections

In order to determine the flow area of the 1D model during the computation, you need cross-sections. These cross-sections are defined by two components: 
1. Cross-section locations
2. Cross-section definitions

A **cross-section location** is defined by the following information:
* The cross-section ID
* A location, specified by a branch ID and a chainage.
* A shift, the vertical shift of the cross section profile. This is mainly used to create a good longitudinal slope over the 1D model (in case of shared cross-sections).
* A reference to the cross-section definition that is used at this location.

<p align="center">
<img alt="Light" src="figures/crosloc.png" width="80%" >
<p\>

A **cross-section definition** is defined by the following information:
* The cross-section definition ID
* The cross-section type: circle, rectangle, ZW river, ZW, YZ, XYZ
* Depending on the type of cross-section, a set of properties describing the properties of the profile shape

<p align="center">
<img alt="Light" src="figures/crsdef.png" width="40%" >
<p\>

In our data folder, there is a CSV file containing the cross-section locations. We want to convert this data to HYDROLIB-core cross-section location objects and add them to the D-Flow FM model. The columns in the CSV correspond exactly with the field names of a `CrossSection` class, making the conversion easy.

**Exercise**
* Read the data from the CSV file
* Print the cross-section location data, to answer the following questions:
  * How many cross-section does the model contain in total, and how many does each branch have?
  * What are the chainages of the cross-sections?
* Convert the CSV data (a pd.DataFrame) into a HYDROLIB-core `CrossSection`

Hint 💡: The MDU file contains a [Geometry] section which has the `crossDefFile` and `crossLocFile` reference, e.g.:

```
[Geometry]
crossDefFile =        # Cross section definitions for all cross section shapes.
crossLocFile =        # Location definitions of the cross sections on a 1D network.
...
```

API references: 
* [CrossLocModel](https://deltares.github.io/HYDROLIB-core/0.5.2/reference/net/#hydrolib.core.dflowfm.net.models.Network)
* [CrossSection](https://deltares.github.io/HYDROLIB-core/0.5.2/reference/net/#hydrolib.core.dflowfm.net.models.NetworkModel)
* [pd.read_csv()](https://pandas.pydata.org/docs/reference/api/pandas.read_csv.html#pandas-read-csv)
* [pd.to_dict()](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.to_dict.html#pandas-dataframe-to-dict)




In [ ]:
# Read profile location file
cross_section_location_file = data_dir / 'cross_section_locations.csv'
cross_section_locations_df = pd.read_csv(cross_section_location_file, index_col=False)

# Answer EX4  - part 1
print(cross_section_locations_df)
# There are 6 cross-sections in the model with 2 cross-sections per branch. 
# The chainage of the cross-sections indicate that the profiles are located upstream and downstream of the branch.

# Create empty cross-section location model
crsloc = CrossLocModel()

# Assign filepath to the model
crsloc.filepath = 'crsloc.ini'

# Convert the dataframe
crsloc.crosssection = cross_section_locations_df.to_dict("records")

# Assign CrossLocModel to the D-Flow FM model.
model.geometry.crosslocfile = crsloc

# Answer EX4 - part 2
print(model.geometry.crosslocfile.crosssection)
print(model.geometry.crosslocfile) # You only get filepath relatively to the MDU-file. 

 



### Add cross-section definitions

In the following cell the definitions belonging to the cross-section locations are added to the D-Flow FM model. These cross-sections are all YZ-profiles in order to define natural shape of the river. 

Please see that the following steps are included:
1. Read the cross-section definitions from an excel. Note that the cross-section definitions id's match with the definitions specified in the crossLocModel.
2. Create an empty [CrossDefModel](https://deltares.github.io/HYDROLIB-core/0.3.1/reference/crosssection/#hydrolib.core.crosssection.models.CrossDefModel) and assign filepath
3. Specify some characteristics for YZ profile. Please consult [appendix C.16.1.6 from D-Flow FM 1D2D](https://content.oss.deltares.nl/delft3dfm1d2d/D-Flow_FM_User_Manual_1D2D.pdf)
4. Create [YZCrsDef](https://deltares.github.io/HYDROLIB-core/0.3.1/reference/crosssection/#hydrolib.core.crosssection.models.YZCrsDef) for each profile definition.
5. Append the definition to the CrossDefModel.
6. Append the CrossDefModel to the D-Flow FM model.


In [ ]:
# Create cross-section definitions
profiles_file = data_dir / '1D_YZ_CrossSections.xlsx'

profiles = pd.read_excel(profiles_file, sheet_name=None)
names = list(profiles.keys())
print(names)

# Create empty CrossDefModel
crsdef = CrossDefModel()
crsdef.filepath = 'crsdef.ini'

# Specify some charachteristics for YZ profile
type_crs = 'yz'
thalweg = 0.0
singleValued = 'yes'
conveyance = 'segmented'
sectionCount = 1

# Create the YZ profiles and append them to the CrossDefModel
for name in names:
    yzCount = len(profiles[name].values)
    ycoord = profiles[name].y.to_list()
    zcoord = profiles[name].z.to_list()
    frictionIds = 'channels'
    frictionPositions = [profiles[name].y[0], profiles[name].y[yzCount-1]]

    crosssection = YZCrsDef(id=name, 
        type=type_crs, 
        thalweg=thalweg, 
        singleValuedZ=singleValued, 
        conveyance=conveyance,
        yzCount=yzCount, 
        yCoordinates= ycoord, 
        zCoordinates=zcoord,
        sectionCount=sectionCount,
        frictionIds = frictionIds,
        frictionPositions=frictionPositions)
    
    crsdef.definition.append(crosssection)
    print(f'definition of {name} added to cross-section model')

# Append CrossDefModel to D-Flow FM model
model.geometry.crossdeffile = crsdef


## Add roughness

The 1D model has now a network topology, computational 1D grid and cross-sections. Friction is a component that should be added to the model to get the proper bed roughness and channel conveyance.

In this step you will add the [FrictionModel](https://deltares.github.io/HYDROLIB-core/0.3.1/reference/friction/#hydrolib.core.friction.models.FrictionModel) to the D-Flow FM model by specifying the friction type and value at the upstream cross-section location (chainage=1000.0 m) of each branch in the model ([FrictBranch](https://deltares.github.io/HYDROLIB-core/0.3.1/reference/friction/#hydrolib.core.friction.models.FrictBranch)). Also a global roughness section is specified ([FrictGlobal](https://deltares.github.io/HYDROLIB-core/0.3.1/reference/friction/#hydrolib.core.friction.models.FrictGlobal)). This global section is used when there is no friction type and values specified for a specific branch. It also contains the frictionId. In the cross-section definition model a reference is made to this frictionId. You can check this with the commands below and search for ```frictionIds```.

In [ ]:
# Print directly the modelobject (in this case all definitions)
print(model.geometry.crossdeffile.definition)

# Or convert it first to a dataframe
df_def = pd.DataFrame([f.__dict__ for f in model.geometry.crossdeffile.definition])
print(df_def[['frictionids']])

Note that YZ profiles can refer to multiple frictionIds, since the friction can vary over the cross-section. Hence the frictionId is given in a ```List()```. This means indirectly that multiple FrictionModels can be added to one D-Flow FM model. The created FrictionModel should therefore been added in a ```List()``` to the D-Flow FM model.

In [ ]:
# Specify frictionId(s) for the FrictionModel(s)
friction_name = 'channels'

# Specify for each FrictionModel a global type and value
global_type = FrictionType.manning # Options are chezy,debosbijkerk, manning, strickler, stricklernikuradse, wallawnikuradse, whitecolebrook
global_value = 0.023

# Create empty friction model and assign filepath
roughness = FrictionModel()
roughness.filepath = "roughness-channels.ini" # often we specify the filenames as "roughness-frictionId.ini"

# Create global friction model and add it to frictionmodel
roughness.global_ = FrictGlobal(frictionId=friction_name,frictionType=global_type,frictionValue=global_value)

# Add friction from branches
function_type = 'constant' # This model uses constant friction per branch

df_friction = branches_gdf[['Name', 'fric_type', 'fric_value', 'chainage']]
df_friction.columns = ['branchId','frictionType','frictionValues', 'chainage']
df_friction = df_friction.assign(frictionValues = [[i] for i in df_friction.frictionValues.values])
df_friction = df_friction.assign(functionType=function_type)
df_friction = df_friction.assign(chainage = [[i] for i in df_friction.chainage.values])
df_friction = df_friction.assign(numlocations=1) # you can specify multiple location along the branch with different friction values.


roughness.branch = df_friction.to_dict('records') # the dataframe is converted to FrictBranch models at once 

# Add frictionmodel to the D-Flow FM model.
model.geometry.frictfile = [roughness] # frictfile is a list of frictionModels

__EX5__: Let's build another frictionmodel with frictionId  _dique_ representing the friction in Canal del Dique. This frictionmodel assumes Chezy as global friction type and has a friction value of 45.

In [ ]:
# Answer to EX5
# Specify frictionId(s) for the FrictionModel(s)
friction_id2 = 'dique'

# Specify for each FrictionModel a global type and value
global_type2 = FrictionType.chezy
global_value2 = 45.0

# Create empty friction model and assign filepath
roughness_dique = FrictionModel()
roughness_dique.filepath = 'roughness-dique.ini'

# Create global friction model and add it to frictionmodel
roughness_dique.global_ = FrictGlobal(frictionId=friction_id2,frictionType=global_type2,frictionValue=global_value2)

# Add frictionmodels to the D-Flow FM model.
model.geometry.frictfile = [roughness, roughness_dique] # Note that the other friction needs to be added to the list.  

The cross-sections on the Canal del Dique (Channel_1D_1) do not refer to this new frictionId. Please uncomment the code below to change this after completion of exercise 5.

In [ ]:
# Convert crosssection defintions from D-Flow FM model to dataframe
df_def = pd.DataFrame([f.__dict__ for f in model.geometry.crossdeffile.definition])
print(df_def)

# Check which cross-section locations are located on Channel_1D_1 
df_loc = pd.DataFrame([f.__dict__ for f in model.geometry.crosslocfile.crosssection])
print(df_loc.loc[:,['id','branchid','definitionid']])

# To which definition do these locations refer:
defs = df_loc.loc[df_loc.index[df_loc.branchid=='Channel_1D_1'],'definitionid'].values

# Change the Frictionids to dique for Channel_1D_1
index_defs = df_def.index[df_def.id.isin(defs)]

for i in index_defs:
    df_def.loc[i, 'frictionids'] = ['dique']

# Overwrite the definitions in D-Flow FM model with the changes in the dataframe
model.geometry.crossdeffile.definition = df_def.to_dict("records")

## Add boundary conditions to the model
Fluvial floods occur when water levels in the river are too high. These high water levels in the rivers can result from several factors. One of those is the occurence of heavy rainfall in the upstream catchment. These heavy rainstorm can lead to a high discharge wave entering the upstream boundary of the river. 

In this tutorial you will model such a high discharge wave (time series) at the upstream 1D boundary (node id = '521410.278872_1056811.761107'). For the two downstream boundaries (node ids = [wkt_geom	Name	BridgeCount	CrossSectio	CulvertCoun	GateCount	GeometryLen	IsLengthCus	LateralSour	Length	LongName	OrderNumber	PumpCount	Source	StructureCo	Target	WeirCount	fric_type	fric_value	chainage]) a constant water level of 0 m AD is assumed. Note that this are tidal boundaries in reality. 

In order to apply boundary conditions on a model you need two components:
1. Forcing (.bc) file [ForcingModel](https://deltares.github.io/HYDROLIB-core/0.3.1/reference/forcing/#hydrolib.core.bc.models.ForcingModel): This model contains the forcing value, time series, astronomical tide, realtime connection with other modules (D-RR) or QHtable. The first two options are created in this tutorial. 
2. External Forcing (.ext) file [ExtModel](https://deltares.github.io/HYDROLIB-core/0.3.1/reference/ext/#hydrolib.core.ext.models.ExtModel): The external forcing specifies the location and type of forcing. Types of forcing are [Boundary](https://deltares.github.io/HYDROLIB-core/0.3.1/reference/ext/#hydrolib.core.ext.models.Boundary), [Lateral](https://deltares.github.io/HYDROLIB-core/0.3.1/reference/ext/#hydrolib.core.ext.models.Lateral) and meteo. 


In [ ]:
# Get node ids of branches
node_ids = model.geometry.netfile.network._mesh1d.network1d_node_id 
print(node_ids)

# Specify the node ids
node_upstream = node_ids[0] # discharge time series node
nodes_downstream = node_ids[2:] # water level nodes

# Create an empty external forcing model and forcing model
ext = ExtModel()
ext.filepath = 'boundary_new.ext'
bc = ForcingModel()
#bc.filepath = inputdir / 'BoundaryConditions.bc'

# Read the upstream boundary time series from excel 
discharge_file = inputdir / '1D_BoundaryConditions.xlsx'
discharge = pd.read_excel(discharge_file, sheet_name=[0])

# Create the timeseries
bc_up = TimeSeries(
    name = node_upstream,
    nodeid = node_upstream,
    timeinterpolation = TimeInterpolation.linear,
    quantityunitpair = [QuantityUnitPair(quantity = 'time', unit = 'minutes since 2012-01-01 00:00:00'), QuantityUnitPair(quantity = 'dischargebnd', unit = 'm³/s')],
    datablock = [[time, dis] for [time,dis] in discharge[0].values]
)

# Create downstream constant boundaries
bc_dn1 = Constant(
    name = nodes_downstream[0],
    nodeid = nodes_downstream[0],
    quantityunitpair = [QuantityUnitPair(quantity = 'waterlevelbnd', unit = 'm')],
    datablock = [["0.0"]]
)

bc_dn2 = Constant(
    name = nodes_downstream[1],
    nodeid = nodes_downstream[1],
    quantityunitpair = [QuantityUnitPair(quantity = 'waterlevelbnd', unit = 'm')],
    datablock = [["0.0"]]
)

# Add the forcing to ForcingModel
bc.forcing.extend([bc_up,bc_dn1,bc_dn2])

# Create boundary locations in ExtModel
ext_up = Boundary(
    quantity = 'dischargebnd',
    nodeid = node_upstream,
    forcingfile = bc
    )

ext_dn1 = Boundary(
    quantity = 'waterlevelbnd',
    nodeid = nodes_downstream[0],
    forcingfile = bc
    )

ext_dn2 = Boundary(
    quantity = 'waterlevelbnd',
    nodeid = nodes_downstream[1],
    forcingfile = bc
    )

# Add the boundaries to ExtModel
ext.boundary.extend([ext_up,ext_dn1,ext_dn2])

# Add the ExtModel to D-Flow FM model
model.external_forcing.extforcefilenew = ext

## Add initial conditions
For this model an initial water depth of 1.0 meter is assumed in the 1D model. 

Below you can see how this initial waterdepth is added to the D-Flow FM model using the following steps:
1. Creating 1D initial conditions field ([1DField](https://deltares.github.io/HYDROLIB-core/0.3.1/reference/onedfield/)), which contains the actual waterdepths on each branch. Note that the file is seperately saved from the rest of the model.
2. Creating [IniFieldModel](https://deltares.github.io/HYDROLIB-core/0.3.1/reference/inifield/) (= Initial and parameter field files) in which we collect the initial condition specified in step 1, so that it can be added to the `FMModel`. 

In [ ]:
# Create 1D initial conditions field
ini_1d = OneDFieldModel()
ini_1d.global_ = OneDFieldGlobal(quantity='waterdepth', unit='m', value=1.0) # only global values are needed in this case, but you could also specify it per branch.
ini_1d.filepath = 'initialwaterdepth.ini'
ini_1d.save(outputdir / ini_1d.filepath) # 1D field ini files needs to be saved seperately from the model save. 

# Create initial field file
ini_field = IniFieldModel()
ini_field.filepath = 'inifield.ini'
ini_field.initial = InitialField(quantity='waterdepth',datafile=ini_1d.filepath, datafiletype=DataFileType.onedfield)
model.geometry.inifieldfile = ini_field


## Add a weir to the model
Many lakes occur close along Canal del Dique. See figure below. This tutorial assumes that these lakes can function as extra storage for the floods from the river. Hence a weir structure, functioning as a small dam, is added downstream of the lakes. This weir structure has the following conditions:
* It is located at 70 km from the upstream node of the branch. 
* The crestlevel is 4.0 m AD.
* The crest width is 105.0 m. 
* The flow over the dam is in both directions. 

<p align="center">
<img alt="Light" src="figures/multiple_lakes.png" width="40%" >
<p\>

In [ ]:
# Create weir structure
weir = Weir(
    id='fictive_dam', 
    name = 'fictive_dam',
    branchId='Channel_1D_1', 
    chainage=70000.0, 
    allowedFlowDir=FlowDirection.both, 
    crestLevel=4.0, 
    crestWidth=105.0, 
    corrCoeff=1.000, 
    useVelocityHeight=True
)

# Create structure model
struc = StructureModel()
struc.filepath = 'structure.ini'

# Append weir to structure
struc.structure.append(weir)

# Assign structure model to D-Flow FM model
model.geometry.structurefile = [struc]



__EX6__: You found out that the dam should be significantly higher. Hence you increase the crestlevel to 10.0 m AD. Your client also decided that flow can only occur from upstream to downstream. Please change the weir above. 

In [ ]:
# Answer EX6: 

# Create weir structure
weir = Weir(
    id='fictive_dam', 
    name = 'fictive_dam',
    branchId='Channel_1D_1', 
    chainage=70000.0, 
    allowedFlowDir=FlowDirection.positive, 
    crestLevel=10.0, 
    crestWidth=105.0, 
    corrCoeff=1.000, 
    useVelocityHeight=True
)

# Create structure model
struc = StructureModel()
struc.filepath = 'structure.ini'

# Append weir to structure
struc.structure.append(weir)

# Assign structure model to D-Flow FM model
model.geometry.structurefile = [struc]

## Save 1D model

The 1D model is now finished. The commands below are used to save the model to the correct filepath (specified for the MDU-file). Note that the ```recurse``` is set to ```True``` to make sure all underlying model files are written to the directory. Else the MDU-file will be the only file written to the directory. 


In [ ]:
# Change some time settings
model.time.refdate = '20120101'
model.time.dtuser = 60. # seconds
model.time.tstart = 43200.
model.time.tstop = 561600.

# Save the 1D model. 
model.general.autostart = AutoStartOption.no # This is a workaround for a bug in hydrolib-core 0.3.1
model.save(recurse=True)

## PART 2: Extend to a 1D2D Delft3D FM model
We will now continue with the model `fm` from part 1, and add a 2D mesh to it, as well as 1D2D couplings.

## Create 2D flexible mesh
We use a shapefile with the area of interest to define the extent of the new 2D mesh that is to be generated.
First, a uniform rectilinear mesh is generated, after which it can be clipped. 

__EX7__: Create a rectangular grid with a cell size of 2500 by 2500 meter. Use [create_rectilinear()](https://deltares.github.io/HYDROLIB-core/0.3.1/reference/net/#hydrolib.core.net.models.Mesh2d.create_rectilinear) function.

__EX8__: Clip the rectangular grid with the ```clip_geom``` using [clip()](https://deltares.github.io/HYDROLIB-core/0.3.1/reference/net/#hydrolib.core.net.models.Mesh2d.clip) function.

In [ ]:
# Read grid outline
gdf_grid = gpd.read_file(inputdir / 'outline_2D_grid.shp')

# Determine the boundary box
extent = gdf_grid.geometry[0].bounds

mesh2d = Mesh2d()
mesh_kernel = mesh2d.meshkernel

xmin, ymin, xmax, ymax = extent

dx=2500.0
dy=2500.0

num_rows = int((ymax - ymin) / dy)
num_columns = int((xmax - xmin) / dx)
origin_x = xmin
origin_y = ymin
block_size_x = dx
block_size_y = dy

make_grid_parameters = MakeGridParameters()
make_grid_parameters.num_columns = num_columns
make_grid_parameters.num_rows = num_rows
make_grid_parameters.origin_x = origin_x
make_grid_parameters.origin_y = origin_y
make_grid_parameters.block_size_x = dx
make_grid_parameters.block_size_y = dy
make_grid_parameters.upper_right_x = xmax
make_grid_parameters.upper_right_y = ymax

# Create uniform rectilinear mesh
mesh_kernel.curvilinear_make_uniform(make_grid_parameters)
mesh_kernel.curvilinear_convert_to_mesh2d()

mesh2d._process(mesh2d.get_mesh2d())

# Clip resulting mesh within polygon area of interest
# Prepare a clipping polygon for MeshKernel input:
clip_polygon = np.array(gdf_grid.geometry[0].exterior.xy)
clip_geom = GeometryList(clip_polygon[0],clip_polygon[1])

# Answer EX8
mesh2d.clip(clip_geom)

model.geometry.netfile.network._mesh2d = mesh2d
model.geometry.netfile.filepath = "FlowFM_1D2D_net.nc"


The cell below produces a basic plot of the 1D + 2D mesh.

In [ ]:
# Create a plot
mesh2d_mk = network._mesh2d.get_mesh2d()
mesh1d_mk = network._mesh1d._get_mesh1d()

figure, ax = plt.subplots()
ax.set_aspect('equal')
plt.rcParams["figure.figsize"]=10,10

ax.imshow(img, extent=[440000,531544.97964175534434617 ,1057000 ,1228644.01383191486820579 ])
# Plot the 1D2D network
mesh2d_mk.plot_edges(ax=ax, color="orange")
mesh1d_mk.plot_edges(ax=ax, color='blue')

### Local refinement of the 2D mesh
One of the key functionalities of Delft3D Flexible Mesh is that you can refine the 2D grid locally. The cell below shows how the grid is locally refined around the river by executing the refinement algorithm once (```level=1```).


In [ ]:
# Refine the coarse uniform grid in a buffer region around the rivers
gdf_refine = gpd.read_file(inputdir / 'buffer_around_river_proj.shp')
refine_polygon = np.array(gdf_refine.geometry[0].exterior.xy)
refine_geom = GeometryList(refine_polygon[0], refine_polygon[1])

parameters = mk.MeshRefinementParameters(
    refine_intersected=True,
    use_mass_center_when_refining=False,
    min_edge_size=10.0,
    refinement_type=1,
    connect_hanging_nodes=True,
    account_for_samples_outside_face=False,
    max_refinement_iterations=1,
)

mesh_kernel.mesh2d_refine_based_on_polygon(refine_geom, parameters)
mesh2d._process(mesh2d.get_mesh2d())
model.geometry.netfile.network._mesh2d = mesh2d
model.geometry.netfile.filepath = "FlowFM_1D2D_refined_net.nc"

In [ ]:
# Create a plot
mesh2d_mk = network._mesh2d.get_mesh2d()
mesh1d_mk = network._mesh1d._get_mesh1d()

figure, ax = plt.subplots()
ax.set_aspect('equal')
plt.rcParams["figure.figsize"]=10,10

ax.imshow(img, extent=[440000,531544.97964175534434617 ,1057000 ,1228644.01383191486820579 ])
# Plot the 1D2D network
mesh2d_mk.plot_edges(ax=ax, color="orange")
mesh1d_mk.plot_edges(ax=ax, color='blue')

## Save 1D2D model
The 1D2D model is now finished. The commands below are used to save the model to the correct filepath (specified for the MDU-file). Note that the ```recurse``` is set to ```True``` to make sure all underlying model files are written to the directory. Else the MDU-file will be the only file written to the directory. 

Note that the 1D2D links will be generated in the Delft3D FM 1D2D software. Future releases of HYDROLIB-core will consist of 1D2D functionalities.

In [ ]:
# Change fm
outputdir_1d2d = root / '1D2D_model'
model.filepath = outputdir_1d2d / 'Magdalena_1D2D.mdu'

model.general.autostart = AutoStartOption.no
model.save(recurse=True)